In [1]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import matplotlib.pyplot as ply
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 5000 

In [2]:
# Load the data online or offline

# ecr_data = web.DataReader('ecr.l', 'av-daily', start=datetime(2020, 1, 1), end=datetime(2021, 9, 24), api_key='EUWPYIBX33PPD5U5')
# data = ecr_data 

# 注意：从csv导入时，index不是‘日期’了。网络加载的data没有这个问题。

data = pd.read_csv(r'/Users/paxton615/Desktop/ECR_csv.csv')

In [3]:
print(data.head(3))
print(data.tail(3))

   Unnamed: 0   open  high     low  close   volume
0  2020-01-02  0.725  0.78  0.6825  0.725   482605
1  2020-01-03  0.725  0.78  0.7150  0.725  1741346
2  2020-01-07  0.670  0.67  0.6690  0.670   309251
     Unnamed: 0   open  high    low  close    volume
393  2021-07-23  1.500  1.60  1.483  1.575   6036634
394  2021-07-26  1.575  1.75  1.565  1.625  11248950
395  2021-07-27  1.650  1.70  1.566  1.600   5443371


- Construct new dataframe, insert placing and warranty data

In [4]:
# Copy data
df = data
ecr = df[['Unnamed: 0','close']]
ecr.columns = ['date','close']
ecr.set_index('date', drop=True, inplace=True)
print(ecr.tail(3))
print('-----------------')
print(len(ecr))

            close
date             
2021-07-23  1.575
2021-07-26  1.625
2021-07-27  1.600
-----------------
396


- Insert 1st Placing data

In [7]:
ecr_p1_sprice = 0.5
ecr_p1_wprice = 1
ecr_p2_sprice = 1.3
ecr_p2_wprice = 2

In [8]:
# 1st placing price is 0.5, date '2020-04-01'
ecr['p1_sprice'] = [ecr_p1_sprice if i  >= '2020-04-01' else None for i in ecr.index]

# 1st placing warranty is 1, date '2020-04-01'
ecr['p1_wprice'] = [ecr_p1_wprice if i  >= '2020-04-01' else None for i in ecr.index]

In [ ]:
# 2nd placing price is 1.3, date '2020-07-01'
ecr['p2_sprice'] = [ecr_p2_sprice if i  >= '2020-07-01' else None for i in ecr.index]

# 2nd placing warranty is 2, date '2020-07-01'
ecr['p2_wprice'] = [ecr_p2_wprice if i  >= '2020-07-01' else None for i in ecr.index]

- If investor put 5000 pounds for each placing, calculate share and warranty positions. 
- Share vs Warranty is 1:1

In [ ]:
ecr['p1_sposition'] = [5000/ecr_p1_sprice if i >= '2020-04-01' else None for i in ecr.index]
ecr['p1_wposition'] = ecr['p1_sposition']


#### 注意：日期千万不能写错'2020-07-04'与'2020-7-04'有天差地别

In [ ]:
ecr['p2_sposition'] = [round(5000/ecr_p2_sprice,0) if i >= '2020-07-01' else None for i in ecr.index]
ecr['p2_wposition'] = ecr['p2_sposition']


- Calculate investor's daily profit per share

#### 注意：股价盈利有盈有亏，但权证是只盈不亏的。

In [ ]:
ecr['p1_spft'] = ecr['close'] - ecr['p1_sprice']
ecr['p1_wpft'] = ecr[]

In [ ]:
# num = 0
# diff = []
# for i,j in zip(ecr['p1_price'],ecr['close']):
#     if i > 0 :
#         diff.append(round(j-i, 4))
#     print(diff)
    

In [ ]:
ecr


In [ ]:
ecr.p1_daypft.max()


In [ ]:
ecr.p2_daypft.max()
